In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
import hvplot.pandas
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
tide_file = '/home/jovyan/data/tide/pred_F.txt'
df_tides = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides['datetime']=df_tides['year'] + '-' + df_tides['month'] + '-' + df_tides['day'] + \
            'T' + df_tides['hour'] + ':' + df_tides['minute'] + ':' + df_tides['second']
#df_tides_E = pd.DataFrame(tides_E)
df_tides.index=pd.to_datetime(df_tides['datetime'].values)
#df_tides_E.index = df_tides_E['parsed_time']
del df_tides['year']
del df_tides['month']
del df_tides['day']
del df_tides['hour']
del df_tides['minute']
del df_tides['second']
del df_tides['datetime']
df_tides['height'] = df_tides['height'].astype(float)
df_tides= df_tides.sort_index()
#df_tides_E['height'] = df_tides_E['height'].to_pandas().resample('T').mean()


In [ ]:
df_tides.head()

In [ ]:
df_tides.describe()

In [ ]:
#list comprehention
epoch= [i.timestamp() for i in df_tides.index.to_pydatetime()]

In [ ]:
df_tides['epoch'] = epoch

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_F11.pkl', 'rb') as F:
    botpt_data = pk.load(F)
df_botpt = pd.DataFrame(botpt_data)
df_botpt['bottom_pressure'] = df_botpt['bottom_pressure'].astype(float)
df_botpt['Depth']=df_botpt['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botpt.index.to_pydatetime()]
df_botpt['epoch'] = epoch
df_botpt= df_botpt.sort_index()
df_botpt.index.name= 'Date'
df_botpt.tail()

In [ ]:
object= df_botpt.hvplot.scatter(x='Date', y='Depth', height=200, datashade=True,
                       flip_yaxis=True)
# hvplot.save(object, 'myname.png')

In [ ]:
df_subsetTides = df_tides.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']
df_subsetBOTPT = df_botpt.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']

In [ ]:
type(df_botpt)

In [ ]:
df_subsetBOTPT['tides'] = df_subsetTides.height
df_subsetBOTPT.tail()

In [ ]:
df_subsetBOTPT['dtide']=  df_subsetBOTPT['Depth'] - df_subsetBOTPT['tides']
df_subsetBOTPT.tail()

In [ ]:
df_subsetBOTPT.hvplot(x='Date', y='dtide', datashade=True, height=200,
                       flip_yaxis=True)

### Import Seismic Data

In [ ]:
!ls '/home/jovyan/data/'

In [ ]:
seismic_file = '/home/jovyan/data/hypo71_2018.dat'
tide_file = '/home/jovyan/data/tide/pred_F.txt'
df_seismic_data = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic_data['datetime'] = df_seismic_data['yyyymmdd'] + 'T' + \
            df_seismic_data['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic_data['HHMM'].str.slice(start=2) 
df_seismic_data.index = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data['datetime'] = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data = df_seismic_data.loc['2017-01-1 00:00:00':'2018-10-31 00:00:00']
del df_seismic_data['yyyymmdd']
del df_seismic_data['HHMM']
del df_seismic_data['SSS.SS']
del df_seismic_data['MW']
del df_seismic_data['NWR']
del df_seismic_data['GAP']
del df_seismic_data['DMIN']
del df_seismic_data['ERH']
del df_seismic_data['ERZ']
del df_seismic_data['ID']